In [2]:
# Import the necessary libraries

# Basic libraries for data manipulation, numerical operations, visualization ect.
import pandas as pd # Import the pandas library for data manipulation and analysis
import numpy as np # Import the numpy library for numerical operations
import seaborn as sns # Import the seaborn library for data visualization based on Matplotlib
import matplotlib.pyplot as plt # Import the pyplot module from matplotlib for creating static plots
import plotly.graph_objects as go  # Import the graph_objects module from plotly for creating interative network plots

# Libraries for Network Analysis and Community Detection
import networkx as nx # import NetworkX for network analysis
from community import community_louvain # Import community detection module from NetworkX

In [3]:
# Define the filepath of the dataset
filepath_edgelist = 'congress.edgelist'

# Use networkx to read the dataset as a DiGraph since the netword is directed Network
data = nx.read_edgelist(filepath_edgelist, create_using = nx.DiGraph)

# Now we convert this edgelist to a pandas dataframe and show it with the .head() function
G_pandas = nx.to_pandas_edgelist(data)

# Convert the 'source' and 'target' column to integers
G_pandas['source'] = G_pandas['source'].astype(int)
G_pandas['target'] = G_pandas['target'].astype(int)

In [4]:
# Define the filepath of the dataset
filepath_username = 'congress_network_data.json'

# Use pandas to read the json-file as a pandas dataframe
username_data = pd.read_json(filepath_username)

# Define a function to extract the username list from a given record
def extract_username_list(record):
    return record

# Apply the 'extract_username_list' function to the 'usernameList' column and create a new column 'Extracted_Username_List'
username_data['Extracted_Username_List'] = username_data['usernameList'].apply(extract_username_list)

# Get the username list from the first record (assuming 'usernameList' is a list of usernames in the DataFrame)
username_list = username_data['usernameList'][0]

# Create a DataFrame from the list of usernames
usernames = pd.DataFrame({'username': username_list})

In [5]:
# After we have made our edgelist to a pandas dataframe we make this to a DiGraph we can use nx.from_pandas_edgelist
G = nx.from_pandas_edgelist(G_pandas,
                            source='source',
                            target='target',
                            edge_attr='weight',
                            create_using = nx.DiGraph)

# Set 'username' as an attribute for each node
attributes = usernames.to_dict()['username']
nx.set_node_attributes(G, attributes, 'username')

In [6]:
centrality_dgr = nx.degree_centrality(G)
dgr = nx.degree(G)
centrality_eig = nx.eigenvector_centrality_numpy(G, weight = 'weight')
centrality_between = nx.betweenness_centrality(G)

In [7]:
dict_dgr = dict(nx.degree(G))
dict_centrality_dgr = dict(nx.degree_centrality(G))
dict_centrality_eig = dict(nx.eigenvector_centrality(G))
dict_centrality_between = dict(nx.betweenness_centrality(G))

In [8]:
nx.set_node_attributes(G, dict_dgr, 'dgr')
nx.set_node_attributes(G, dict_centrality_dgr, 'centrality_dgr')
nx.set_node_attributes(G, dict_centrality_eig, 'centrality_eig')
nx.set_node_attributes(G, dict_centrality_between, 'centrality_between')

In [12]:
top = 1
nodes_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
top_central_nodes = nodes_df[nodes_df.centrality_eig > nodes_df.centrality_eig.quantile(1-top)].index
G_sub = nx.subgraph(G, top_central_nodes)

In [13]:
# We convert our directed network into an undirected one
G_und = nx.to_undirected(G_sub)
partition = community_louvain.best_partition(G_und)
nx.set_node_attributes(G_und, partition, 'community')
nodes_df_und = pd.DataFrame.from_dict(dict(G_und.nodes(data=True)), orient='index')

In [14]:
from pyvis.network import Network
import pyvis
import networkx as nx

net = Network(
    notebook=True, 
    cdn_resources="remote",
    select_menu=True,
    filter_menu=True,
    )

net.set_options("""
const options = {
  "physics": {
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
"""             
)

net.from_nx(G_und)

In [15]:
net.show('nx.html')

nx.html
